In [1]:
import pandas as pd
import geopandas as gpd


from requests import request
from math import ceil
from shapely.geometry import MultiPoint

from IPython.display import Javascript, display
from ipywidgets import widgets
from ipyleaflet import Map, GeoData, basemaps, LayersControl


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
base_url = 'https://idpgis.ncep.noaa.gov/arcgis/rest/services/NOS_Observations/CO_OPS_Stations/MapServer/1/'
# base_url = 'https://idpgis.ncep.noaa.gov/arcgis/rest/services/NOS_Observations/CO_OPS_Stations/MapServer/2/'

In [3]:
oid_url = base_url +'query?where=objectId>0&returnGeometry=false&returnIdsOnly=true&f=json'
response = request(method='GET',
                   url=oid_url)

assert response.ok
num_features = len(response.json()['objectIds'])
max_features = ceil(num_features / 100) * 100
print("Total number of features in this dataset: {0}".format(num_features))

Total number of features in this dataset: 2723


In [4]:
chunk_size = 100
geojson_features_list = []

i = 0
while i <= max_features:
    oid_min = i
    oid_max = i + chunk_size
    new_url = base_url + 'query?&where=objectid>={0}+and+objectid<={1}&outFields=*&f=geojson'.format(oid_min, 
                                                                                         oid_max)
    response = request(method='GET', 
                       url=new_url)
    assert response.ok

    features = response.json()['features']
    geojson_features_list += features
    i += chunk_size
    print("Successfully returned OIDs {0}:{1}".format(oid_min,
                                                      oid_max))

#We are hard coding in the EPSG:3857 WGS coordinate reference system here. 
tide_station_gdf = gpd.GeoDataFrame.from_features(geojson_features_list, 
                                             crs='epsg:4326')

del geojson_features_list

Successfully returned OIDs 0:100
Successfully returned OIDs 100:200
Successfully returned OIDs 200:300
Successfully returned OIDs 300:400
Successfully returned OIDs 400:500
Successfully returned OIDs 500:600
Successfully returned OIDs 600:700
Successfully returned OIDs 700:800
Successfully returned OIDs 800:900
Successfully returned OIDs 900:1000
Successfully returned OIDs 1000:1100
Successfully returned OIDs 1100:1200
Successfully returned OIDs 1200:1300
Successfully returned OIDs 1300:1400
Successfully returned OIDs 1400:1500
Successfully returned OIDs 1500:1600
Successfully returned OIDs 1600:1700
Successfully returned OIDs 1700:1800
Successfully returned OIDs 1800:1900
Successfully returned OIDs 1900:2000
Successfully returned OIDs 2000:2100
Successfully returned OIDs 2100:2200
Successfully returned OIDs 2200:2300
Successfully returned OIDs 2300:2400
Successfully returned OIDs 2400:2500
Successfully returned OIDs 2500:2600
Successfully returned OIDs 2600:2700
Successfully returned 

In [5]:
tide_station_gdf.iloc[0]['data']

'https://tidesandcurrents.noaa.gov/inventory.html?id=1495000'

In [6]:
def run_next_cell(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
def onclick(event, feature, properties, id):
    print(feature['properties']['id'])
    print(feature)

In [7]:
center = MultiPoint(tide_station_gdf['geometry'].values).centroid
m = Map(center=(center.y, center.x), zoom=3, basemap=basemaps.Esri.WorldTopoMap)
tide_data = GeoData(geo_dataframe=tide_station_gdf, 
                    style={'color':'purple', 
                           'opacity':3},
                   name='Tide Stations')
tide_data.on_click(onclick)
m.add_layer(tide_data)
m

Map(center=[37.41027937345032, -93.3070639065408], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [8]:
button = widgets.Button(description = 'Run Cell')
button.on_click(run_next_cell)
display(button)

Button(description='Run Cell', style=ButtonStyle())